<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s Overal Message:
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [18]:
import tensorflow as tf
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [19]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-08-31 18:44:51--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-08-31 18:44:51 (52.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [20]:
from helper_functions import unzip_data, plot_loss_curves, compare_historys

In [21]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-08-31 18:44:51--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-08-31 18:44:51 (105 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [22]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [23]:
train_data = pd.read_csv('train.csv')
train_data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [24]:
train_data['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [25]:
test_data = pd.read_csv('test.csv')
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [26]:
random_data = train_data.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
lmao fuckboy changed his @ for damage control
@Pseudojuuzo : This is NOT Disaster tweet
------------------------------------

tweet number 2 :
It still hasn't sunk in that i will never see my nan again how has it been 2 months already ?? : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
'Up to 40% of businesses affected by a natural or man-made disaster never reopen'
http://t.co/35JyAp0ul9 : This is NOT Disaster tweet
------------------------------------

tweet number 4 :
[Tropical Cyclone Info] SOUDELOR 945hPa maximum wind speed: 45m/s maximum wind gust speed: 60m/s  http://t.co/nBD5oT9iEW : This is a Disaster tweet
------------------------------------

tweet number 5 :
Businesses are deluged with invoices. Make yours stand oup with colour or shame and it's likely to rise to the top of the pay' pile. : This is NOT Disaster tweet
------------------------------------



## Extract validation data from train data:

In [27]:
train_data['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [28]:
train_data['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [29]:
train_data, val_data, train_label, val_label = train_test_split(train_data['text'].to_numpy(), train_data['target'].to_numpy(), 
                                                                test_size=0.1, 
                                                                random_state=42)

In [30]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [31]:
sum([len(tweet.split()) for tweet in train_data])

101905

In [32]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [33]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [34]:
text_vectorizer.adapt(train_data)

In [35]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorizer(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  707,    9,    1,    8,  171,  714,    5, 1815,   12,    0,
          0,    0,    0,    0])>

In [36]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorizer(random_tweet))

tweet:  Interview on The Collapse of Materialism Best #TalkRadio Listen Live: http://t.co/sDXZHjco0X 


vectorized tweet:  tf.Tensor(
[1828   11    2  165    6    1  148 4460  842  186    1    0    0    0
    0], shape=(15,), dtype=int64)


In [37]:
#unique words
unique_words = text_vectorizer.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['palm', 'palinfoen', 'palestinian\x89Û', 'palermo', 'pale', 'pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']


## Embedding:

In [38]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim =32, input_length=None)

In [39]:
sample_embed = embedding(text_vectorizer(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[ 4.95999493e-02,  2.97597311e-02,  2.24883221e-02,
         1.37032382e-02,  2.39667930e-02,  1.45406984e-02,
        -4.70464714e-02,  3.61466072e-02,  2.55289115e-02,
         1.48401372e-02, -2.57436763e-02,  4.47167046e-02,
         4.65462320e-02,  1.52932890e-02, -3.88342366e-02,
        -9.98225063e-03,  2.78122462e-02,  8.73101875e-03,
         3.65645550e-02,  3.92185561e-02, -2.10557468e-02,
         3.86008061e-02, -1.88783407e-02,  4.21723612e-02,
         4.59686257e-02,  3.12895440e-02,  4.08907272e-02,
         1.74560584e-02, -4.35197949e-02,  2.79454626e-02,
        -2.30152495e-02, -1.59051046e-02],
       [ 4.35223319e-02, -4.37743329e-02, -1.17691644e-02,
        -2.22696420e-02, -4.44970392e-02,  1.81947611e-02,
         1.08673088e-02,  1.65772438e-03,  3.03240865e-03,
        -3.54013592e-03,  6.43665716e-03,  4.68684919e-02,
        -1.38645880e-02, -3.33461985e-02, -4.04698141e-02,
        -3.08910757e-0

## Analyze with Naive Bayes using TF-IDF:

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [100]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [101]:
val_pred = model_0.predict(val_data)
val_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [102]:
val_label[:10]

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 1])

In [103]:
accuracy_score(val_label, val_pred)

0.7782152230971129

In [104]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.74      0.92      0.82       426
           1       0.86      0.60      0.70       336

    accuracy                           0.78       762
   macro avg       0.80      0.76      0.76       762
weighted avg       0.79      0.78      0.77       762



In [105]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [106]:
model_0.predict_proba(tweets)

array([[0.22063312, 0.77936688],
       [0.58453661, 0.41546339],
       [0.71572258, 0.28427742]])

In [107]:
model_0.predict(tweets)

array([1, 0, 0])

## Analyze with Vanilla Neural Network:

In [43]:
from helper_functions import create_tensorboard_callback

In [109]:
inputs = tf.keras.layers.Input(shape= (1,), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.keras.layers.Dense(10, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs, name='ANN')

In [110]:
model_1.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 global_average_pooling1d_1   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 10)                330       
                                                                 
 dense_5 (Dense)             (None, 1)                 11      

In [111]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [112]:
train_label.shape

(6851,)

In [113]:
train_data.shape

(6851,)

In [114]:
# !rm -rf './tensorboard/ANN'

In [115]:
history_1 = model_1.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220831-141651
Epoch 1/10
215/215 [==============================] - 2s 8ms/step - loss: 0.6208 - accuracy: 0.6679 - val_loss: 0.5361 - val_accuracy: 0.7730
Epoch 2/10
215/215 [==============================] - 2s 7ms/step - loss: 0.4213 - accuracy: 0.8285 - val_loss: 0.4756 - val_accuracy: 0.7887
Epoch 3/10
215/215 [==============================] - 2s 7ms/step - loss: 0.3107 - accuracy: 0.8786 - val_loss: 0.4865 - val_accuracy: 0.7900
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.2415 - accuracy: 0.9099 - val_loss: 0.5231 - val_accuracy: 0.7782
Epoch 5/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1920 - accuracy: 0.9295 - val_loss: 0.5659 - val_accuracy: 0.7795
Epoch 6/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1538 - accuracy: 0.9435 - val_loss: 0.6151 - val_accuracy: 0.7546
Epoch 7/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1244 - 

In [116]:
model_1.evaluate(val_data,val_label)

24/24 [==============================] - 0s 2ms/step - loss: 0.8066 - accuracy: 0.7375


[0.8065906167030334, 0.7375327944755554]

In [117]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [118]:
model_1.predict(tweets)

array([[0.99008334],
       [0.8733567 ],
       [0.13929242]], dtype=float32)

## Retrieve the trained word embeddings:

In [120]:
weights = model_1.get_layer('embedding_1').get_weights()[0]
weights

array([[-0.08298621, -0.13634099,  0.05477232, ...,  0.04899124,
        -0.02966163,  0.2385307 ],
       [-0.05942355, -0.17479044,  0.05854312, ..., -0.01682679,
        -0.08783812,  0.3249058 ],
       [-0.11072761, -0.2244753 ,  0.02839673, ...,  0.00070712,
        -0.03209427,  0.3251321 ],
       ...,
       [ 0.04783612, -0.09144634, -0.04565182, ...,  0.06586561,
         0.01778056, -0.02519452],
       [-0.03338373,  0.00361751,  0.03419486, ..., -0.03272738,
        -0.0116438 ,  0.01218984],
       [-0.01321663, -0.0235847 , -0.01725016, ...,  0.02444207,
        -0.04261788, -0.00392542]], dtype=float32)

In [121]:
weights.shape

(10000, 32)

In [122]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [123]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [124]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Then, use [Embedding Projector](http://projector.tensorflow.org/) to Visualize the embeddings

## Analyzie with LSTM model:

In [131]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.LSTM(64,  return_sequences=True)(x)
x = tf.keras.layers.LSTM(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs, name='LSTM')

In [132]:
model_2.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 lstm_7 (LSTM)               (None, 15, 64)            24832     
                                                                 
 lstm_8 (LSTM)               (None, 32)                12416     
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                              

In [133]:
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [134]:
# !rm -rf './tensorboard/LSTM'

In [135]:
history_2 = model_2.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','LSTM')])

Saving TensorBoard log files to: tensorboard/LSTM/20220831-142049
Epoch 1/10
215/215 [==============================] - 11s 32ms/step - loss: 0.1675 - accuracy: 0.9540 - val_loss: 0.8968 - val_accuracy: 0.7454
Epoch 2/10
215/215 [==============================] - 7s 32ms/step - loss: 0.0629 - accuracy: 0.9755 - val_loss: 1.2763 - val_accuracy: 0.7559
Epoch 3/10
215/215 [==============================] - 6s 28ms/step - loss: 0.0532 - accuracy: 0.9768 - val_loss: 1.0775 - val_accuracy: 0.7480
Epoch 4/10
215/215 [==============================] - 6s 28ms/step - loss: 0.0490 - accuracy: 0.9781 - val_loss: 1.2846 - val_accuracy: 0.7349
Epoch 5/10
215/215 [==============================] - 6s 28ms/step - loss: 0.0466 - accuracy: 0.9791 - val_loss: 1.4607 - val_accuracy: 0.7507
Epoch 6/10
215/215 [==============================] - 6s 28ms/step - loss: 0.0455 - accuracy: 0.9800 - val_loss: 1.1833 - val_accuracy: 0.7441
Epoch 7/10
215/215 [==============================] - 6s 28ms/step - loss: 

In [136]:
model_2.evaluate(val_data,val_label)

24/24 [==============================] - 0s 12ms/step - loss: 1.5811 - accuracy: 0.7402


[1.5810790061950684, 0.7401574850082397]

### custom tweets:

In [137]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [138]:
model_2.predict(tweets)

array([[9.9982065e-01],
       [9.6496427e-01],
       [8.6528063e-04]], dtype=float32)

## Analyzie with GRU model:

In [139]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.GRU(64,  return_sequences=True)(x)
x = tf.keras.layers.GRU(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs, name='GRU')

In [140]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 gru (GRU)                   (None, 15, 64)            18816     
                                                                 
 gru_1 (GRU)                 (None, 32)                9408      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                               

In [141]:
model_3.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [142]:
# !rm -rf './tensorboard/RNN'

In [143]:
history_3 = model_3.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','GRU')])

Saving TensorBoard log files to: tensorboard/GRU/20220831-142217
Epoch 1/10
215/215 [==============================] - 69s 33ms/step - loss: 0.1454 - accuracy: 0.9475 - val_loss: 0.8878 - val_accuracy: 0.7336
Epoch 2/10
215/215 [==============================] - 6s 27ms/step - loss: 0.0575 - accuracy: 0.9804 - val_loss: 1.1885 - val_accuracy: 0.7402
Epoch 3/10
215/215 [==============================] - 6s 28ms/step - loss: 0.0389 - accuracy: 0.9787 - val_loss: 1.7085 - val_accuracy: 0.7073
Epoch 4/10
215/215 [==============================] - 7s 32ms/step - loss: 0.0386 - accuracy: 0.9809 - val_loss: 1.6198 - val_accuracy: 0.7388
Epoch 5/10
215/215 [==============================] - 6s 27ms/step - loss: 0.0436 - accuracy: 0.9799 - val_loss: 1.6255 - val_accuracy: 0.7297
Epoch 6/10
215/215 [==============================] - 6s 28ms/step - loss: 0.0414 - accuracy: 0.9799 - val_loss: 1.7360 - val_accuracy: 0.7454
Epoch 7/10
215/215 [==============================] - 6s 28ms/step - loss: 0

In [144]:
model_3.evaluate(val_data,val_label)

24/24 [==============================] - 0s 6ms/step - loss: 1.6278 - accuracy: 0.7402


[1.6277616024017334, 0.7401574850082397]

### custom tweets:

In [145]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [146]:
model_3.predict(tweets)

array([[9.9919200e-01],
       [8.8962078e-01],
       [3.2281876e-04]], dtype=float32)

## Analyzie with Bidirectional LSTM model:

In [147]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)


x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

In [148]:
model_4.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                

In [149]:
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [150]:
# !rm -rf './tensorboard/RNN'

In [151]:
history_4 = model_4.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-142423
Epoch 1/10
215/215 [==============================] - 19s 50ms/step - loss: 0.1151 - accuracy: 0.9677 - val_loss: 1.0156 - val_accuracy: 0.7598
Epoch 2/10
215/215 [==============================] - 9s 41ms/step - loss: 0.0395 - accuracy: 0.9809 - val_loss: 1.4057 - val_accuracy: 0.7441
Epoch 3/10
215/215 [==============================] - 9s 40ms/step - loss: 0.0381 - accuracy: 0.9804 - val_loss: 1.4385 - val_accuracy: 0.7100
Epoch 4/10
215/215 [==============================] - 9s 40ms/step - loss: 0.0394 - accuracy: 0.9781 - val_loss: 1.6245 - val_accuracy: 0.7441
Epoch 5/10
215/215 [==============================] - 9s 41ms/step - loss: 0.0316 - accuracy: 0.9841 - val_loss: 1.7084 - val_accuracy: 0.7336
Epoch 6/10
215/215 [==============================] - 9s 40ms/step - loss: 0.0379 - accuracy: 0.9809 - val_loss: 1.7069 - val_accuracy: 0.7375
Epoch 7/10
215/215 [==============================] - 9s 40ms

In [152]:
model_4.evaluate(val_data,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 1.8066 - accuracy: 0.7323


[1.8065708875656128, 0.7322834730148315]

### custom tweets:

In [153]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [154]:
model_4.predict(tweets)

array([[9.9964130e-01],
       [1.6864109e-01],
       [9.6925622e-05]], dtype=float32)

## Analyzie with Convolutional 1D model:

In [155]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)
print(x.shape)

x = tf.keras.layers.Conv1D(filters=64, kernel_size=3,padding= 'valid')(x)
print(x.shape)
x = tf.keras.layers.Conv1D(filters=128, kernel_size=3,padding= 'valid')(x)
print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

(None, 15, 32)
(None, 13, 64)
(None, 11, 128)


In [156]:
model_5.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 conv1d (Conv1D)             (None, 13, 64)            6208      
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 128)           24704     
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                       

In [157]:
model_5.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [158]:
# !rm -rf './tensorboard/RNN'

In [159]:
history_5 = model_5.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-142655
Epoch 1/10
215/215 [==============================] - 4s 12ms/step - loss: 0.1495 - accuracy: 0.9521 - val_loss: 0.9122 - val_accuracy: 0.7415
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0649 - accuracy: 0.9743 - val_loss: 1.1226 - val_accuracy: 0.7349
Epoch 3/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0474 - accuracy: 0.9785 - val_loss: 1.2601 - val_accuracy: 0.7178
Epoch 4/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0425 - accuracy: 0.9784 - val_loss: 1.5800 - val_accuracy: 0.7349
Epoch 5/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0403 - accuracy: 0.9796 - val_loss: 1.7032 - val_accuracy: 0.7297
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0383 - accuracy: 0.9794 - val_loss: 2.0785 - val_accuracy: 0.7257
Epoch 7/10
215/215 [==============================] - 3s 12ms/

In [160]:
model_5.evaluate(val_data,val_label)

24/24 [==============================] - 0s 4ms/step - loss: 3.0524 - accuracy: 0.7087


[3.052351236343384, 0.7086614370346069]

### custom tweets:

In [161]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [162]:
model_5.predict(tweets)

array([[0.99969554],
       [0.99963874],
       [0.91976   ]], dtype=float32)

## Analyzie with Universal Sentence Encoder tensorflow hub:

In [163]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed(["The quick brown fox jumps over the lazy dog.",
                      "I am a sentence for which I would like to get its embedding"])

print(embed_sample)

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.05080861 -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [40]:
embed(train_data)

<tf.Tensor: shape=(6851, 512), dtype=float32, numpy=
array([[ 0.01178656, -0.00748094,  0.00952065, ...,  0.04002216,
        -0.07293098,  0.01045   ],
       [ 0.08340861, -0.03587737,  0.01278975, ..., -0.00764408,
        -0.06952015, -0.08435119],
       [ 0.03161547,  0.01725737, -0.00808266, ..., -0.03697908,
         0.03299483, -0.04934364],
       ...,
       [-0.03393723, -0.06167451,  0.04978931, ..., -0.05219645,
        -0.05812604, -0.0630734 ],
       [ 0.0215332 ,  0.06060665, -0.07061633, ..., -0.04428503,
        -0.00652964, -0.00850323],
       [ 0.02649508, -0.05326023, -0.05051778, ..., -0.01227468,
         0.01156288, -0.00953933]], dtype=float32)>

In [46]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               trainable=False,
                               dtype=tf.string,
                               name= 'USE')

In [14]:
model_6 = tf.keras.Sequential([
    encoder_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')],
    name = 'USE_pretrained_model'
)

In [45]:
model_6.summary()

Model: "USE_pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_9 (Dense)             (None, 64)                32832     
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 34,945
Non-trainable params: 256,797,824
_________________________________________________________________


In [16]:
model_6.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [169]:
# !rm -rf './tensorboard/RNN'

In [44]:
history_6 = model_6.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','USE_pretrained_model')])

Saving TensorBoard log files to: tensorboard/USE_pretrained_model/20220831-184621
Epoch 1/10
215/215 [==============================] - 7s 16ms/step - loss: 0.4940 - accuracy: 0.7882 - val_loss: 0.4201 - val_accuracy: 0.8150
Epoch 2/10
215/215 [==============================] - 3s 13ms/step - loss: 0.4057 - accuracy: 0.8229 - val_loss: 0.4039 - val_accuracy: 0.8189
Epoch 3/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3873 - accuracy: 0.8348 - val_loss: 0.4065 - val_accuracy: 0.8189
Epoch 4/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3694 - accuracy: 0.8431 - val_loss: 0.4046 - val_accuracy: 0.8176
Epoch 5/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3491 - accuracy: 0.8514 - val_loss: 0.4171 - val_accuracy: 0.8097
Epoch 6/10
215/215 [==============================] - 3s 13ms/step - loss: 0.3260 - accuracy: 0.8662 - val_loss: 0.4213 - val_accuracy: 0.8110
Epoch 7/10
215/215 [==============================] - 3s 13m

In [171]:
model_6.evaluate(val_data,val_label)

24/24 [==============================] - 0s 13ms/step - loss: 0.4851 - accuracy: 0.8241


[0.48508840799331665, 0.8241469860076904]

### custom tweets:

In [172]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [173]:
model_6.predict(tweets)

array([[0.9989565 ],
       [0.9247216 ],
       [0.00365987]], dtype=float32)

## Finding most wrong predicted tweets:

In [47]:
train_data

array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
       'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
       "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
       ...,
       'Omron HEM-712C Automatic Blood Pressure Monitor STANDARD AND LARGE BP CUFFS http://t.co/gJBAInQWN9 http://t.co/jPhgpL1c5x',
       'Officials say a quarantine is in place at an Alabama home over a possible Ebola case after developing symptoms... http://t.co/rqKK15uhEY',
       'I moved to England five years ago today. What a whirlwind of time it has been! http://t.co/eaSlGeA1B7'],
      dtype=object)

In [48]:
train_label

array([0, 1, 0, ..., 0, 1, 1])

In [59]:
import numpy as np
pred_probs = np.squeeze(model_6.predict(train_data))
print(pred_probs)
pred_label = np.squeeze(np.round(pred_probs))
pred_label

[0.22332045 0.9854951  0.00905359 ... 0.02063134 0.99930537 0.83901286]


array([0., 1., 0., ..., 0., 1., 1.], dtype=float32)

In [62]:
import pandas as pd
comparison_table = pd.DataFrame({'tweets':train_data,
              'target': train_label,
              'pred': pred_label,
              'prob': pred_probs})

comparison_table

,tweets,target,pred,prob
0,'McFadden Reportedly to Test Hamstring Thursda...,0,0.0,0.223320
1,w--=-=-=-[ NEMA warns Nigerians to prepare for...,1,1.0,0.985495
2,When I was cooking earlier I got electrocuted ...,0,0.0,0.009054
3,I'm On Fire. http://t.co/WATsmxYTVa,0,0.0,0.105393
4,More than 40 families affected by the fatal ou...,1,1.0,0.921615
...,...,...,...,...
6846,@Eganator2000 There aren't many Obliteration s...,0,0.0,0.165271
6847,just had a panic attack bc I don't have enough...,0,0.0,0.058754
6848,Omron HEM-712C Automatic Blood Pressure Monito...,0,0.0,0.020631
6849,Officials say a quarantine is in place at an A...,1,1.0,0.999305


In [68]:
worst_preds = comparison_table[comparison_table['target'] != comparison_table['pred']].sort_values('prob')
worst_preds

,tweets,target,pred,prob
5277,Oh the usual. Mass murder and world domination...,1,0.0,0.018337
3445,You can never escape me. Bullets don't harm me...,1,0.0,0.020957
366,going to redo my nails and watch behind the sc...,1,0.0,0.022397
5799,I liked a @YouTube video http://t.co/43sXG9Z6x...,1,0.0,0.027035
1772,So I pick myself off the ground and swam befor...,1,0.0,0.027519
...,...,...,...,...
5859,.POTUS #StrategicPatience is a strategy for #G...,0,1.0,0.879290
4653,#hot C-130 specially modified to land in a st...,0,1.0,0.897060
4648,wowo--=== 12000 Nigerian refugees repatriated ...,0,1.0,0.958887
5582,Kosciusko police investigating pedestrian fata...,0,1.0,0.991505


In [70]:
worst_preds[:10]

,tweets,target,pred,prob
5277,Oh the usual. Mass murder and world domination...,1,0.0,0.018337
3445,You can never escape me. Bullets don't harm me...,1,0.0,0.020957
366,going to redo my nails and watch behind the sc...,1,0.0,0.022397
5799,I liked a @YouTube video http://t.co/43sXG9Z6x...,1,0.0,0.027035
1772,So I pick myself off the ground and swam befor...,1,0.0,0.027519
3498,Emergency Flow http://t.co/lH9mrYpDrJ mp3 htt...,1,0.0,0.030804
3415,@accionempresa ChinaÛªs stock market crash th...,1,0.0,0.032136
4970,Black Eye 9: A space battle occurred at Star M...,1,0.0,0.033065
5014,?? New Ladies Shoulder Tote #Handbag Faux Leat...,1,0.0,0.034967
6085,my dad said I look thinner than usual but real...,1,0.0,0.040242


In [71]:
worst_preds[-10:]

,tweets,target,pred,prob
6277,Trafford Centre film fans angry after Odeon ci...,0,1.0,0.751855
799,Two Jewish Terrorists Charged In Historic-Chur...,0,1.0,0.759550
5890,Teen Disaster Preparedness Event in Van Nuys A...,0,1.0,0.803780
5208,PawSox owners public return from whirlwind tri...,0,1.0,0.836698
243,@adorableappple No reported flooding po in the...,0,1.0,0.844792
5859,.POTUS #StrategicPatience is a strategy for #G...,0,1.0,0.879290
4653,#hot C-130 specially modified to land in a st...,0,1.0,0.897060
4648,wowo--=== 12000 Nigerian refugees repatriated ...,0,1.0,0.958887
5582,Kosciusko police investigating pedestrian fata...,0,1.0,0.991505
6548,DISASTER AVERTED: Police kill gunman with Û÷h...,0,1.0,0.992520


### 5 most wrong disaster tweets:


In [108]:
for i in np.arange(0,5):
  print(worst_preds.iloc[i,0],'\n')
  print(f'target: {worst_preds.iloc[i,1]}, prediction: {worst_preds.iloc[i,2]}, prob: {worst_preds.iloc[i,3]} \n')
  print('========================')

Oh the usual. Mass murder and world domination plans over coffee. How's your day going? 

target: 1, prediction: 0.0, prob: 0.018337130546569824 

You can never escape me. Bullets don't harm me. Nothing harms me. But I know pain. I know pain. Sometimes I share it. With someone like you. 

target: 1, prediction: 0.0, prob: 0.02095693349838257 

going to redo my nails and watch behind the scenes of desolation of smaug ayyy 

target: 1, prediction: 0.0, prob: 0.022396713495254517 

I liked a @YouTube video http://t.co/43sXG9Z6xh TREMOR IS NO JOKE!! [TREMOR DLC] [FATALITIES/X-RAY] 

target: 1, prediction: 0.0, prob: 0.02703458070755005 

So I pick myself off the ground and swam before I drowned. Hit the bottom so hard I bounced twice suffice this time around is different. 

target: 1, prediction: 0.0, prob: 0.027518808841705322 



### 5 most wrong Non-disaster tweets:


In [105]:
for i in np.arange(1,6)*-1:
  print(worst_preds.iloc[i,0],'\n')
  print(f'target: {worst_preds.iloc[i,1]}, prediction: {worst_preds.iloc[i,2]}, prob: {worst_preds.iloc[i,3]} \n')
  print('========================')

DISASTER AVERTED: Police kill gunman with Û÷hoax deviceÛª atåÊcinema http://t.co/5NG0FzpVdS 

target: 0, prediction: 1.0, prob: 0.9925195574760437 

Kosciusko police investigating pedestrian fatality hit by a train Thursday http://t.co/m5djLLxoZP 

target: 0, prediction: 1.0, prob: 0.9915049076080322 

wowo--=== 12000 Nigerian refugees repatriated from Cameroon 

target: 0, prediction: 1.0, prob: 0.958886981010437 

#hot  C-130 specially modified to land in a stadium and rescue hostages in Iran in 1980 http://t.co/zY3hpdJNwg #prebreak #best 

target: 0, prediction: 1.0, prob: 0.8970597982406616 

.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4 

target: 0, prediction: 1.0, prob: 0.8792896270751953 

